# Chapter 01: LLM Fine-tuning 개요

## 1. 학습 목표

* LLM Fine-tuning의 개념과 필요성을 명확히 이해한다.
* Base Model과 Finetuned Model의 차이를 파악한다.
* Fine-tuning이 적합한 상황과 고려해야 할 한계를 인식한다.
* gpt-oss-20b 모델의 사양을 확인하고 메모리 요구량을 추정한다.

## 2. LLM Fine-tuning이란?

### 2.1 정의

LLM(Large Language Model)은 대규모 텍스트 데이터로 사전 학습된 언어 모델이다. **Fine-tuning**은 이러한 사전 학습된 모델(Pre-trained Model)을 특정 태스크나 도메인 데이터셋을 사용하여 추가 학습시키는 과정이다. 이를 통해 범용 모델을 특정 목적에 최적화된 전문가 모델로 변환할 수 있다.

```text
Fine-tuning = 사전학습 모델 + 특화 데이터셋 학습 → 특화된 모델

```

### 2.2 왜 Fine-tuning이 필요한가?

사전학습 모델(Base Model)은 방대한 지식을 가지고 있지만, 실제 서비스에 바로 적용하기에는 다음과 같은 한계가 있다.

1. **명령어 추종 부족**: 사용자의 복잡한 지시사항을 정확히 따르지 못하거나 횡설수설하는 경향이 있다.
2. **도메인 지식 부족**: 의료, 법률, 금융 등 특수 분야의 최신 전문 용어나 뉘앙스를 이해하지 못한다.
3. **일관성 부재**: 대화 스타일이나 톤이 일정하지 않아 브랜드 페르소나를 유지하기 어렵다.
4. **안전성 문제**: 편향되거나 유해한 콘텐츠 생성을 효과적으로 제어하기 어렵다.

Fine-tuning은 이러한 문제를 해결하여 모델을 사용자의 의도에 맞게 정렬(Align)하는 핵심 과정이다.

## 3. Base Model vs Finetuned Model

모델은 일반적으로 다음 세 단계를 거쳐 발전한다.

1. **Pre-training (사전학습)**
* **목적**: 언어의 문법, 문맥, 일반 상식을 이해하고 생성하는 능력 습득.
* **데이터**: 수조 토큰 이상의 인터넷 텍스트 데이터 (비지도 학습).
* **결과물**: Base Model (예: gpt-oss-20b-base). 문장을 이어 쓰는 능력은 뛰어나지만 질문에 답하는 능력은 부족할 수 있다.


2. **Supervised Fine-tuning (SFT)**
* **목적**: 명령어(Instruction)를 이해하고 적절히 반응하는 능력 부여.
* **데이터**: (명령어, 응답) 쌍으로 구성된 고품질 데이터셋.
* **결과물**: Instruction-tuned Model. 사용자의 요청에 맞춰 작업을 수행한다.


3. **Alignment (정렬)**
* **목적**: 인간의 가치관과 선호도에 부합하는 안전하고 유용한 응답 생성.
* **데이터**: 선호도 비교 데이터 (Chosen vs Rejected).
* **방법**: RLHF(Reinforcement Learning from Human Feedback), DPO(Direct Preference Optimization) 등.
* **결과물**: Chat Model (예: gpt-oss-20b-chat).

## 4. Fine-tuning이 적합한 상황

모든 상황에 Fine-tuning이 필요한 것은 아니다. RAG(Retrieval-Augmented Generation)나 프롬프트 엔지니어링으로 해결 가능한지 먼저 검토해야 한다.

### Fine-tuning이 필수적인 경우

* **페르소나 부여**: 특정 캐릭터, 말투, 성격을 일관되게 유지해야 하는 챗봇이나 NPC를 개발할 때.
* **전문 용어 및 패턴 학습**: 사내 문서 양식, 특수 코딩 스타일, 의학적 소견서 작성 등 고유한 형식을 따라야 할 때.
* **엄격한 규칙 적용**: "항상 JSON 형식으로만 출력하라"와 같은 강한 제약 조건을 안정적으로 준수해야 할 때.
* **지연 시간(Latency) 및 비용 최적화**: 복잡한 프롬프트(Few-shot examples)를 매번 입력하는 대신, 모델 자체에 지식을 내재화하여 추론 토큰 수를 줄이고 속도를 높일 때.

## 5. gpt-oss-20b 모델 및 실습 준비

본 튜토리얼에서는 가상의 오픈소스 모델인 `gpt-oss-20b`를 기준으로 설명한다. 이는 약 200억 개의 파라미터를 가진 모델로, 고성능이면서도 소비자용 하드웨어(또는 소규모 클라우드 GPU)에서 효율적인 학습 기법을 실습하기에 적합한 크기를 상정한 것이다. 

### 실습: 모델 정보 확인 및 메모리 추정

먼저 필요한 라이브러리를 임포트하고 모델의 설정(Config)을 확인하여 구조를 파악한다. 그 후, 정밀도(Precision)에 따른 메모리 요구량을 계산해본다.

In [1]:
# pytorch 설치 - 설치된 CUDA 버전에 맞게끔 설치한다.
# https://pytorch.org/get-started/previous-versions/?_gl=1*zquwyp*_up*MQ..*_ga*MTI4Mjk5MDcwNy4xNzY4MTQzNTA5*_ga_469Y0W5V62*czE3NjgxNDM1MDgkbzEkZzEkdDE3NjgxNDM1MTIkajU2JGwwJGgw

In [2]:
%pip install -q transformers

Note: you may need to restart the kernel to use updated packages.


In [1]:
# 필수 라이브러리 임포트
import torch
from transformers import AutoConfig

# 실습 대상 모델 설정
model_name = "openai/gpt-oss-20b"

print(f"분석 대상 모델: {model_name}")

/home/josh/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


분석 대상 모델: openai/gpt-oss-20b


In [2]:
# 모델 설정(Config) 로드
# 예외 처리를 제거하고 직접 로드한다.
# 모델에 접근할 수 없는 환경이라면 HuggingFace 토큰 설정이 선행되어야 한다.
config = AutoConfig.from_pretrained(model_name)

print("=" * 40)
print(f"모델 아키텍처 정보")
print("=" * 40)
print(f"모델 유형: {config.model_type}")
print(f"히든 사이즈 (Hidden Size): {config.hidden_size}")
print(f"레이어 수 (Num Layers): {config.num_hidden_layers}")
print(f"어텐션 헤드 수 (Num Heads): {config.num_attention_heads}")
print(f"어휘 크기 (Vocab Size): {config.vocab_size}")
# max_position_embeddings 속성이 없는 경우를 대비해 getattr 대신 직접 확인하거나 기본값을 가정할 수 있으나,
# 최신 모델은 대부분 해당 속성을 가진다.
print(f"최대 컨텍스트 길이: {config.max_position_embeddings}")
print("=" * 40)

모델 아키텍처 정보
모델 유형: gpt_oss
히든 사이즈 (Hidden Size): 2880
레이어 수 (Num Layers): 24
어텐션 헤드 수 (Num Heads): 64
어휘 크기 (Vocab Size): 201088
최대 컨텍스트 길이: 131072


### 메모리 요구량 계산 함수

Fine-tuning을 위해 필요한 VRAM 용량을 추정하는 유틸리티 함수를 구현한다. 파라미터 수와 정밀도(FP32, FP16, INT8, INT4)에 따라 필요한 메모리는 크게 달라진다.

In [3]:
def estimate_model_memory(num_params_billion, precision="fp16"):
    """
    모델 파라미터 수와 정밀도에 따른 메모리 요구량을 추정하는 함수다.

    Args:
        num_params_billion (float): 파라미터 수 (단위: 십억/Billion)
        precision (str): 정밀도 ('fp32', 'fp16', 'bf16', 'int8', 'int4')

    Returns:
        dict: 추론 및 학습 시 예상 메모리 사용량 (GB)
    """
    # 정밀도별 파라미터당 바이트 수
    bytes_per_param = {
        "fp32": 4,    # 32-bit floating point
        "fp16": 2,    # 16-bit floating point
        "bf16": 2,    # 16-bit bfloat
        "int8": 1,    # 8-bit integer
        "int4": 0.5   # 4-bit integer
    }

    # 지원하지 않는 정밀도가 입력되면 기본값(fp16) 사용
    param_bytes = bytes_per_param.get(precision, 2)

    # 1. 모델 가중치 로드에 필요한 메모리 (GB)
    # 10억 * 바이트 수
    model_memory_gb = num_params_billion * param_bytes

    # 2. 학습 시 추가 메모리 오버헤드 추정
    # Full Fine-tuning의 경우 Optimizer 상태(AdamW 등)와 Gradient 저장을 위해
    # 모델 크기의 약 3~4배가 추가로 필요하다.
    # QLoRA 등 PEFT 방식은 이 오버헤드가 현저히 적다. 여기서는 단순화된 계수를 사용한다.
    if precision in ["fp32", "fp16", "bf16"]:
        # Full Fine-tuning 가정 (가중치 + 그라디언트 + 옵티마이저 상태)
        training_overhead = 4.0
    else:
        # PEFT/QLoRA 가정 (가중치는 고정, 어댑터만 학습하므로 오버헤드 적음)
        training_overhead = 1.3

    training_memory_gb = model_memory_gb * training_overhead

    return {
        "precision": precision,
        "inference_memory_gb": round(model_memory_gb, 2),
        "training_memory_gb": round(training_memory_gb, 2)
    }

# gpt-oss-20b (20B 파라미터) 기준 메모리 추정
target_params = 20 # 20 Billion

print(f"\n[{target_params}B 모델 메모리 요구량 추정]")
print("-" * 60)
print(f"{'Precision':<10} | {'Inference (GB)':<15} | {'Training (Est. GB)':<15}")
print("-" * 60)

for prec in ["fp16", "int8", "int4"]:
    mem_info = estimate_model_memory(target_params, prec)
    print(f"{mem_info['precision']:<10} | {mem_info['inference_memory_gb']:<15} | {mem_info['training_memory_gb']:<15}")
print("-" * 60)
print("* Training 메모리는 학습 방법(Full vs LoRA)과 배치 크기에 따라 크게 달라질 수 있다.")


[20B 모델 메모리 요구량 추정]
------------------------------------------------------------
Precision  | Inference (GB)  | Training (Est. GB)
------------------------------------------------------------
fp16       | 40              | 160.0          
int8       | 20              | 26.0           
int4       | 10.0            | 13.0           
------------------------------------------------------------
* Training 메모리는 학습 방법(Full vs LoRA)과 배치 크기에 따라 크게 달라질 수 있다.


## 6. 요약

이 챕터에서는 LLM Fine-tuning의 기본 개념과 필요성을 학습했다.
Base Model을 사용자의 목적에 맞는 모델로 변환하기 위해 SFT와 Alignment 과정이 필요함을 이해했다.
또한, 20B 규모의 모델을 다루기 위해서는 상당한 VRAM이 필요하며, 이를 해결하기 위해 정밀도를 낮추는 양자화(Quantization)와 효율적인 학습 방법이 필수적임을 메모리 추정 실습을 통해 확인했다.

다음 챕터에서는 이러한 메모리 제약을 극복하고 효율적으로 학습할 수 있는 구체적인 방법론인 **Fine-tuning 유형 및 방법론(LoRA, QLoRA 등)**에 대해 상세히 다룬다.